# College Basketball Data Analysis

Doug Beick | Udacity Data Scientist Nanodegree\
April, 2021

The goal in this project is to take data from multiple seasons of college basketball to assess team and conference performance, both in season and postseason.

---
## 2_clean_prepare

Using pre-sourced data on NCAA men's basketball performance for years 2013-2019, this notebook
builds a single large data source, including the addition of four columns for later use.

This notebook needs only be run once to build the dataset. Since we export to csv, later analysis
should already have the dataset ready.

### Helpers

In [1]:
import pandas as pd

#Dictionary denoting how many teams equalled/excelled a given team's postseason performance, inclusive
postseason_encode_dict = {
    'Champions': 1,
    '2ND': 2,
    'F4': 4,
    'E8': 8,
    'S16': 16,
    'R32': 32,
    'R64': 64,
    'R68': 68,
}

#Dictionary of [extra games, extra wins] a team earned, based on postseason performance
tourney_g_w_dict = {
    'Champions': [6, 6],
    '2ND': [6, 5],
    'F4': [5, 4],
    'E8': [4, 3],
    'S16': [3, 2],
    'R32': [2, 1],
    'R64': [1, 0],
    'R68': [1, 0],
}

#Ordering for columns
col_ordered = ['TEAM', 'CONF', 'G', 'W', 'PRE_PC', 'POST_PC', 'ADJOE', 'ADJDE', 'BARTHAG', 'EFG_O', 'EFG_D',
       'TOR', 'TORD', 'ORB', 'DRB', 'FTR', 'FTRD', '2P_O', '2P_D', '3P_O',
       '3P_D', 'ADJ_T', 'WAB', 'POSTSEASON', 'SEED', 'PFPC', 'YEAR']

### Clean and prepare data

In [2]:
df_dict = {}
for i in range(3,10):
    df = pd.read_csv('datasets/cbb1{0}.csv'.format(i))
    team_count = len(df)
    
    #Column denoting season
    df['YEAR'] = '201{0}'.format(i)
    
    #Column denoting performance percentile of the team, based on postseason finish
    df['PFPC'] = df.apply(lambda row: round((team_count+1 - postseason_encode_dict[row.POSTSEASON])/team_count, 4)
                                  if not pd.isnull(row.POSTSEASON)
                                  else 0, axis = 1)
    
    #Column denoting win percentage of team, prior to postseason tournament
    df['PRE_PC'] = df.apply(lambda row: round((row.W - tourney_g_w_dict[row.POSTSEASON][1])
                                                            /(row.G - tourney_g_w_dict[row.POSTSEASON][0]), 4)
                                               if not pd.isnull(row.POSTSEASON)
                                               else round((row.W / row.G), 4), axis = 1)
    
    #Column denoting win percentage of team, following postseason tournament
    df['POST_PC'] = df.apply(lambda row: round((row.W/row.G), 4), axis = 1)
    
    #Invert column out of 100, to properly reflect defensive rate, instead of previous representation
    #  which was offensive rate allowed
    df['DRB'] = df.apply(lambda row: 100 - row.DRB, axis=1)
    
    #Fix inconsistent labels in CONF column
    df['CONF'] = df.apply(lambda row: row.CONF.capitalize()
                         if row.CONF == 'ind'
                         else row.CONF, axis = 1)
    
    df = df[col_ordered]
    df_dict["csv1{0}".format(i)] = df
     
dfAll = pd.concat(df_dict.values()).sort_values(by=['YEAR', 'PFPC', 'SEED', 'W'], ascending=[True, False, True, False])

### Export to csv

In [3]:
dfAll.to_csv('datasets/cbbAll.csv', index=False)